# 딥러닝 학습방법 이해하기
- 지난번까지는 데이터를 선형 모델로 해석하는 방법을 배웠다면 이제는 비선형모델인 신경망을 알아본다.

신경망을 수식으로 분해하려면 먼저 선형모델을 이해해야 한다.

각 행벡터는 데이터와 가중치 행렬 사이의 행렬곱과 절편벡터의 합으로 표현된다고 가정하자.
- 이 경우 데이터가 바귀면 결과값도 바뀌게 된다. 이 때 출력 벡터의 차원 또한 바뀌게 된다.

이 방법으로 출력벡터에 softmax 함수를 합성하면 확률벡터가 되며 특정 클래스에 속할 확률로 해석할 수 있다.


## 소프트맥스 연산
- 소프트맥스(softmax)함수는 모델의 출력을 확률로 해석할 수 있게 변환해주는 연산
- 분류 문제를 풀 때 선형모델과 소프트맥스 함수를 결합하여 예측한다

In [2]:
import numpy as np
def softmax(vec) :
    denumerator = np.exp(vec - np.max(vec, axis = 1, keepdims = True))
    numerator = np.sum(denumerator, axis = 1, keepdims = True)
    val = denumerator / numerator
    return val

vec = np.array([[1, 2, 0], [-1, 0, 1], [-10, 0, 10]])
softmax(vec)

array([[2.44728471e-01, 6.65240956e-01, 9.00305732e-02],
       [9.00305732e-02, 2.44728471e-01, 6.65240956e-01],
       [2.06106005e-09, 4.53978686e-05, 9.99954600e-01]])

In [3]:
# 추론을 할 경우 one-hot 벡터로 최대값을 가진 주소만 1로 출력하는 연산을 사용하여
# softmax를 사용하는 경우가 드물다
def one_hot(val, dim) :
    return [np.eye(dim)[_] for _ in val]

def one_hot_encoding(vec) :
    vec_dim = vec.shape[1]
    vec_argmax = np.argmax(vec, axis = 1)
    return one_hot(vec_argmax, vec_dim)

def softmax(vec) :
    denumerator = np.exp(vec - np.max(vec, axis = 1, keepdims = True))
    numerator = np.sum(denumerator, axis = 1, keepdims = True)
    val = denumerator / numerator
    return val

vec = np.array([[1, 2, 0], [-1, 0, 1], [-10, 0, 10]])
print(one_hot_encoding(vec))
print(one_hot_encoding(softmax(vec)))

[array([0., 1., 0.]), array([0., 0., 1.]), array([0., 0., 1.])]
[array([0., 1., 0.]), array([0., 0., 1.]), array([0., 0., 1.])]


## 활성함수(Activation Function)
신경망은 선형모델과 활성함수를 합성한 함수이다
- 활성함수는 실수 위에 정의된 비선형 함수로서 딥러닝에서 매우 중요한 개념이다.
- 활성함수를 쓰지 않으면 딥러닝은 선형모델과 차이가 없다.
- 시그모이드 함수나 tanh 함수는 전통적으로 많이 쓰이던 활성함수지만 딥러닝에선 ReLU 함수를 많이 쓰고 있다.


## 신경망을 수식으로 분해해보자
- 다층 퍼셉트론은 신경망이 여러층 합성된 함수이다.

이론적으로는 2층 신경망으로도 임의의 연속함수를 근사할 수 있다.

그러나 층을 여러개 쌓는 이유는
- 층이 깊을수록 목적함수를 근사하는데 필요한 뉴런(노드)의 숫자가 훨씬 빨리 줄어들어 좀 더 효율적으로 학습이 가능하기 때문이다.
    - 얇은 층이라면 필요한 뉴런의 숫자가 기하급수적으로 늘어 넓은 신경망이 되어야 한다.

## 딥러닝 학습 원리 - 역전파 알고리즘
- 딥러닝은 역전파(backpropagation) 알고리즘을 이용하여 각 층에 사용된 패러미터를 학습한다.
- 각 층 패러미터의 그레디언트 벡터는 윗층부터 역순으로 계산하게 된다.
- 역전파 알고리즘은 합성함수의 미분법인 연쇄법칙 기반 자동미분을 사용한다.
    - 각 노드의 텐서값을 컴퓨터가 기억해야 미분계산이 가능해진다.